<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>

# 为大规模推理部署模型

## 04 - 简单的 TensorFlow 模型
-------

**目录**

* [简介](#introduction)
* [创建模型目录结构](#structure)
* [定义简单的 TensorFlow 模型](#model)
* [创建配置文件](#configuration)
* [在 Triton 推理服务器中加载模型](#load)
* [将推理请求发送到服务器](#infer)
* [练习](#exercise)
* [小结](#conclusion)


<a id="introduction"></a>
### 简介

在本笔记本中，我们将创建一个 TensorFlow ResNet 50 模型，然后以 `SavedModel` 的表示形式保存起来，并使用 Triton 推理服务器进行部署。 我们将看到如何在 Triton 推理服务器中创建模型目录结构和配置文件，如何使用 TensorFlow，以及如何向 Triton 推理服务器中部署的模型发送推理请求。

<a id="structure"></a>
### 创建模型目录结构

Triton 推理服务器可为模型库中的模型提供服务。首次运行 Triton 推理服务器时，您需要指定模型所在的模型库位置：

```
tritonserver --model-repository=/models
```

每个模型都位于模型库内对应的模型子目录中，即 `/models` 中的每个目录均表示一个唯一的模型。例如，我们将在此 notebook 中部署 TensorFlow 模型 (`simple-TensorFlow-model`)。

所有模型通常遵循相似的目录结构。在每个目录中，我们将创建配置文件 `config.pbtxt`，用它来详细介绍模型信息，例如批量大小、输入形状、部署后端（比如 PyTorch、ONNX、TensorFlow、TensorRT）等等。稍后，我们将在此 notebook 中探索配置文件。

此外，我们还可以创建一个或多个模型版本。每个版本都位于具有相应版本号（始于 `1`）的子目录名称之下。配置文件位于我们模型文件所在的此子目录内（例如 `model.onnx`、`model.savedmodel`）。

```
root@server:/models$ tree
.
├── simple-tensorflow-model
│   ├── 1
│   │   └── model.savedmodel
│   │       ├── assets
│   │       ├── saved_model.pb
│   │       └── variables
│   │           ├── variables.data-00000-of-00001
│   │           └── variables.index

```

我们还可以添加一个文件以表示输出端的名称。为简洁起见，我们已在此 notebook 中省略了此步骤。如需详细了解如何在 Triton 推理服务器中使用模型库和模型目录结构，请在此参阅以下文档：https://github.com/triton-inference-server/server/blob/r20.12/docs/model_repository.md

下面，我们将为每个 TensorFlow 模型创建模型目录结构。

In [ ]:
!mkdir -p models/simple-tensorflow-model/
!mkdir -p models/simple-tensorflow-model/1/

<a id="model"></a>
### 定义简单的 TensorFlow 模型

下面，我们将定义简单的 TensorFlow ResNet50 模型。我们将指定使用预训练的模型，即以 ImageNet 上训练时学到的权重来实例化 ResNet50 模型。定义了类 `WrappedModel` 后，我们将实例化此模型，使用私有的 `__call__` 方法获取调用签名，然后与该调用签名一起使用 `tf.saved_model.save` 函数来保存模型。

我们将在 `simple-tensorflow-model` 模型目录的版本 `1` 子目录下，以 `SavedModel` 表示形式导出模型。

In [ ]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)


class WrappedModel(tf.Module):
    def __init__(self):
        super(WrappedModel, self).__init__()
        self.model = tf.keras.applications.ResNet50()
    @tf.function
    def __call__(self, x):
        return self.model(x)

model = WrappedModel()
call = model.__call__.get_concrete_function(tf.TensorSpec([None, None, None, None], 
                                            tf.float32, name='input_0'))
tf.saved_model.save(model, 
                    'models/simple-tensorflow-model/1/model.savedmodel', 
                    signatures=call)

接下来，加载 ImageNet 标签。

In [ ]:
import json

with open('./imagenet-simple-labels.json') as file:
    labels = json.load(file)

print(labels[:5])

使用 Triton 推理服务器之前，我们需要确认在 ImageNet 上预训练的 ResNet50 模型适用于样本图像。我们将使用金鱼图像，您可以随意尝试使用自己的图像！

In [ ]:
import numpy as np
from PIL import Image


img_path = './assets/goldfish.jpg'
image_pil = Image.open(img_path)
image_pil

接下来，我们将从 TensorFlow Keras 中导入一个 ResNet50 基础模型，以及一些用于加载图像、调整图像大小、预处理输入和解码预测的辅助函数。我们发现模型如我们预期一般，可以正确分类金鱼图像。

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img = image.load_img(img_path, target_size=(224, 224))
image_numpy = image.img_to_array(img)
image_numpy = np.expand_dims(image_numpy, axis=0)
image_numpy = preprocess_input(image_numpy)

preds = model.predict(image_numpy)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

<a id="configuration"></a>
### 创建配置文件

以 `SavedModel` 表示形式定义并写入模型后，我们现在将注意力转向为模型创建配置文件。

对于模型配置而言，至少须指定模型名称、平台或后端属性、max_batch_size 属性以及模型的输入和输出张量（名称、数据类型和形状）。


如需详细了解如何在 Triton 推理服务器中创建模型配置文件，请参阅相关文档：
https://github.com/triton-inference-server/server/blob/r20.12/docs/model_configuration.md

In [ ]:
configuration = """
name: "simple-tensorflow-model"
platform: "tensorflow_savedmodel"
max_batch_size: 32
input [
 {
    name: "input_0"
    data_type: TYPE_FP32
    format: FORMAT_NHWC
    dims: [ 224, 224, 3 ]
  }
]
output {
    name: "output_0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""

with open('models/simple-tensorflow-model/config.pbtxt', 'w') as file:
    file.write(configuration)

<a id="load"></a>
### 在 Triton 推理服务器中加载模型


创建模型目录结构、定义和导出模型以及创建配置文件后，我们现在将等待 Triton 推理服务器来加载模型。我们设置此实验以在**轮询**模式下使用 Triton 推理服务器。这意味着 Triton 推理服务器将以 30 秒为间隔，持续轮询模型的修改内容或新创建的模型。请运行以下单元，以预留一些时间，以便 Triton 推理服务器对新模型/修改内容进行轮询，然后再继续下一步操作。

In [ ]:
!sleep 45

此时，我们的模型应已部署就绪且随时可用！为确认 Triton 推理服务器已启动并运行，我们会看到对以下 URL 的 `curl` 请求。

In [ ]:
!curl -v triton:8000/v2/health/ready

如果 Triton 已准备就绪，则 HTTP 请求会返回状态 200；如果 Triton 未准备就绪，则会返回 200 以外的状态。

我们还可以向模型端点发送 `curl` 请求，以确认我们的模型已部署就绪并可随时使用。如果模型已准备就绪，此 `curl` 请求会返回状态 200；如果模型未准备就绪，则会返回 200 以外的状态。

此外，我们还将看到模型的相关信息，例如：

* 模型的名称、
* 模型可用的版本、
* 后端平台（例如 tensorflow_savedmodel）、
* 附带各自名称、数据类型和形状的输入与输出。


In [ ]:
!curl -v triton:8000/v2/models/simple-tensorflow-model

<a id="infer"></a>
### 将推理请求发送到服务器

模型部署就绪后，即可向模型发送推理请求。

首先，我们将加载 `tritonclient.http` 模组和实用程序函数，用于处理 NumPy 数据。


In [ ]:
import tritonclient.http as tritonhttpclient
from tritonclient.utils import triton_to_np_dtype

接下来，我们将定义模型的输入和输出名称、模型名称、使用 Triton 推理服务器向其中部署模型的 URL（本例中为 `triton:8000` 的本地主机）以及模型版本。

In [ ]:
VERBOSE = False
input_name = 'input_0'
input_shape = (1, 224, 224, 3)
input_dtype = 'FP32'
output_name = 'output_0'
model_name = 'simple-tensorflow-model'
url = 'triton:8000'
model_version = '1'

我们将使用 `tritonhttpclient.InferenceServerClient` 类通过 `.get_model_metadata()` 方法访问模型元数据，并使用 `get_model_config()` 方法获取模型配置，进而实例化客户端。

In [ ]:
triton_client = tritonhttpclient.InferenceServerClient(url=url, verbose=VERBOSE)
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)
model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

使用预期输入名称、形状和数据类型来实例化输入数据的占位符。将金鱼图像的输入数据设置为 NumPy 数组表示形式。还需仅使用输出名称实例化输出数据的占位符。

最后，我们将使用 `triton_client.infer()` 方法将输入提交至 Triton 推理服务器，指定模型名称、模型版本、输入和输出，并将结果转换为 NumPy 数组。

In [ ]:
input0 = tritonhttpclient.InferInput(input_name, input_shape, input_dtype)
input0.set_data_from_numpy(image_numpy, binary_data=False)

output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=False)
response = triton_client.infer(model_name, model_version=model_version, 
                               inputs=[input0], outputs=[output])
logits = response.as_numpy(output_name)
logits = np.asarray(logits, dtype=np.float32)

所需的全部操作如上所述！我们可以识别最大的 logit 值，并确认我们的模型正确推断出图像实际上是一条金鱼。

In [ ]:
print(labels[np.argmax(logits)])

<a id="conclusion"></a>
### 小结

我们在此 notebook 中展示了如何创建 TensorFlow ResNet50 模型，以 `SavedModel` 的表示形式加以保存，然后使用 Triton 推理服务器加以部署。我们了解了如何在 Triton 推理服务器中创建模型目录结构和配置文件、如何使用 TensorFlow，以及如何向 Triton 推理服务器中部署的模型发送推理请求。

我们建议您运行下面的单元进行清理。此操作将释放 GPU 显存，以供实验的其他部分使用。

In [ ]:
!rm -rf models/simple-tensorflow-model

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>